# ГлавРосГосНефть

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
# Импортируем библиотеки pandas
import numpy as np
import pandas as pd

# И все используемое далее
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
try:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    ! gdown --id 1dm9Di_rTM6Y_vMStKtr0Db_7uU71973w
    df = pd.read_csv('/content/geo_data_0.csv')

In [ ]:
try:
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    ! gdown --id 1GBZuhsiovVDZT-XqN1zqBHuSWIB2JeZL
    df = pd.read_csv('/content/geo_data_1.csv')

In [ ]:
try:
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    ! gdown --id 1GaKWhALkVO9xD-eH4H_fAUtORlMuT7ln
    df = pd.read_csv('/content/geo_data_2.csv')

In [ ]:
# Убираем длинные циферки
pd.set_option('display.float_format', '{:,.2f}'.format)

# Смотрим таблички
display(df_0.columns)
print()
display(df_0)

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04
...,...,...,...,...,...
99995,DLsed,0.97,0.37,6.08,110.74
99996,QKivN,1.39,-0.38,1.27,122.35
99997,3rnvd,1.03,0.02,-1.35,64.38
99998,7kl59,1.00,-0.53,1.58,74.04


In [ ]:
display(df_1.columns)
print()
display(df_1)

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77
...,...,...,...,...,...
99995,QywKC,9.54,-6.88,2.00,53.91
99996,ptvty,-10.16,-12.56,5.01,137.95
99997,09gWa,-7.38,-3.08,5.00,137.95
99998,rqwUm,0.67,-6.15,1.00,30.13


In [ ]:
display(df_2.columns)
print()
display(df_2)

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60
...,...,...,...,...,...
99995,4GxBu,-1.78,1.13,6.26,172.33
99996,YKFjq,-1.26,-0.89,2.52,138.75
99997,tKPY3,-1.20,-2.96,5.22,157.08
99998,nmxp2,-2.42,2.42,-5.55,51.80


### Инфо. Пропуски. Describe.

In [ ]:
# Посмотрим заполненость таблицы df_0
print('\nИнфо:')
print(df_0.info())                             # сколько значений и в каком формате
print('\nПропусков:')
print(df_0.isna().mean())                      # подсчёт пропусков в долях (перевод в % - лишний код)
print('\ndescribe:')
display(df_0.describe())                       # вдруг чего "на глазок" проявится 
print('\nДубликатов =', df_0.duplicated().sum()) # проверим дубликаты


Инфо:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Пропусков:
id        0.00
f0        0.00
f1        0.00
f2        0.00
product   0.00
dtype: float64

describe:


,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.72,128.56
max,2.36,1.34,16.00,185.36



Дубликатов = 0


In [ ]:
# Посмотрим заполненость таблицы df_1
print('\nИнфо:')
print(df_1.info())
print('\nПропусков:')
print(df_1.isna().mean())
print('\ndescribe:')
display(df_1.describe())
print('\nДубликатов =', df_1.duplicated().sum())


Инфо:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Пропусков:
id        0.00
f0        0.00
f1        0.00
f2        0.00
product   0.00
dtype: float64

describe:


,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,1.14,-4.80,2.49,68.83
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95



Дубликатов = 0


In [ ]:
# Посмотрим заполненость таблицы df_2
print('\nИнфо:')
print(df_2.info())
print('\nПропусков:')
print(df_2.isna().mean())
print('\ndescribe:')
display(df_2.describe())
print('\nДубликатов =', df_2.duplicated().sum())


Инфо:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Пропусков:
id        0.00
f0        0.00
f1        0.00
f2        0.00
product   0.00
dtype: float64

describe:


,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03



Дубликатов = 0


In [ ]:
# Поищем артефакты в значениях:
display('f0 в df_0:', df_0['f0'].sort_values().unique())

'f0 в df_0:'

array([-1.40860531, -1.35177299, -1.30222711, ...,  2.33375269,
        2.33707957,  2.36233081])

In [ ]:
display('f1 в df_0:', df_0['f1'].sort_values().unique())

'f1 в df_0:'

array([-0.8482185 , -0.84490792, -0.8205609 , ...,  1.33334561,
        1.33482762,  1.34376933])

In [ ]:
display('f2 в df_0:', df_0['f2'].sort_values().unique())

'f2 в df_0:'

array([-12.08832812, -10.13834135, -10.13817115, ...,  15.23032159,
        15.42837187,  16.00379001])

In [ ]:
display('product в df_0:', df_0['product'].sort_values().unique())

'product в df_0:'

array([0.00000000e+00, 4.02152316e-03, 6.11363631e-03, ...,
       1.85355615e+02, 1.85362690e+02, 1.85364347e+02])

In [ ]:
display('f0 в df_1:', df_1['f0'].sort_values().unique())

'f0 в df_1:'

array([-31.60957602, -27.82961614, -26.64625507, ...,  28.93082879,
        29.25906208,  29.42175461])

In [ ]:
display('f1 в df_1:', df_1['f1'].sort_values().unique())

'f1 в df_1:'

array([-26.35859801, -25.38962242, -25.2915177 , ...,  16.0268693 ,
        16.7371962 ,  18.73406263])

In [ ]:
display('f2 в df_1:', df_1['f2'].sort_values().unique())

'f2 в df_1:'

array([-0.01814409, -0.01788668, -0.01768626, ...,  5.01750345,
        5.01909142,  5.01972056])

In [ ]:
display('product в df_1:', df_1['product'].sort_values().unique())

'product в df_1:'

array([  0.        ,   3.17910258,  26.95326103,  30.13236361,
        53.90652206,  57.08562465,  80.85978309,  84.03888568,
       107.81304413, 110.99214671, 134.76630516, 137.94540774])

In [ ]:
display('f0 в df_2:', df_2['f0'].sort_values().unique())

'f0 в df_2:'

array([-8.76000362, -7.45058711, -7.18949804, ...,  7.19461485,
        7.21552717,  7.23826248])

In [ ]:
display('f1 в df_2:', df_2['f1'].sort_values().unique())

'f1 в df_2:'

array([-7.08401976, -6.74835677, -6.73299712, ...,  7.10161842,
        7.76185714,  7.84480127])

In [ ]:
display('f2 в df_2:', df_2['f2'].sort_values().unique())

'f2 в df_2:'

array([-11.97033454, -11.61169048, -11.40724351, ...,  16.31301122,
        16.35764509,  16.73940206])

In [ ]:
display('product в df_2:', df_2['product'].sort_values().unique())

'product в df_2:'

array([0.00000000e+00, 4.60600004e-03, 9.20411196e-03, ...,
       1.90011722e+02, 1.90013589e+02, 1.90029838e+02])

### Описание данных

	• id — уникальный идентификатор скважины;
	• f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
	• product — объём запасов в скважине (тыс. баррелей).


### Условия задачи

	• Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
	• При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
	• Бюджет на разработку скважин в регионе — 10 млрд рублей.
	• При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
	• После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

##### Вывод

- Количество записей по 100000.
- Пропусков данных нет.
- Дубликатов не имеется.
- Артефактов в данных не обнаружено.

## Обучение и проверка модели для каждого региона

- [x]  2.1. Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25.
- [x]  2.2. Обучаем модель и делааем предсказания на валидационной выборке.
- [x]  2.3. Сохраняем предсказания и правильные ответы на валидационной выборке.
- [x]  2.4. Выводим на экране средний запас предсказанного сырья и RMSE модели.
- [x]  2.5. Анализируем результаты.

In [ ]:
# Делим таблички на train (75%), valid (25%)

features_0 = df_0.drop(['product', 'id'], axis=1)
target_0 = df_0['product']  # будем прогнозировать объём запасов в скважине
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
                                                                        features_0, target_0, 
                                                                        test_size=0.25, 
                                                                        random_state=123)
features_1 = df_1.drop(['product', 'id'], axis=1)
target_1 = df_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
                                                                        features_1, target_1, 
                                                                        test_size=0.25, 
                                                                        random_state=123)
features_2 = df_2.drop(['product', 'id'], axis=1)
target_2 = df_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
                                                                        features_2, target_2, 
                                                                        test_size=0.25, 
                                                                        random_state=123)

In [ ]:
# Линейная регрессия

model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

result_0 = (mean_squared_error(target_valid_0, predictions_valid_0))** (0.5)
scores_0 = cross_val_score(model_0, features_0, target_0, cv=5)
final_score_0 = final_score = sum(scores_0) / len(scores_0)   # cредняя оценка качества модели

df_valid_0 = pd.DataFrame(zip(target_valid_0))                # target и predictions в табличку для дальнейшей работы
df_valid_0 = df_valid_0.rename(columns={0: 'target_valid_0'}) # меняем дурное название столбца
df_valid_0["predictions_valid_0"] = predictions_valid_0

print("\nСредний запас предсказанного сырья:", float('{:.3f}'.format(predictions_valid_0.mean())))
print("RMSE модели model_0:", float('{:.3f}'.format(result_0)))
print("Средняя оценка качества модели:", float('{:.3f}'.format(final_score_0)))

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

result_1 = (mean_squared_error(target_valid_1, predictions_valid_1))** (0.5)
scores_1 = cross_val_score(model_1, features_1, target_1, cv=5)
final_score_1 = final_score = sum(scores_1) / len(scores_1)

df_valid_1 = pd.DataFrame(zip(target_valid_1))
df_valid_1 = df_valid_1.rename(columns={0: 'target_valid_1'})
df_valid_1["predictions_valid_1"] = predictions_valid_1

print("\nСредний запас предсказанного сырья:", float('{:.3f}'.format(predictions_valid_1.mean())))
print("RMSE модели model_1:", float('{:.3f}'.format(result_1)))
print("Средняя оценка качества модели:", float('{:.4f}'.format(final_score_1)))

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

result_2 = (mean_squared_error(target_valid_2, predictions_valid_2))** (0.5)
scores_2 = cross_val_score(model_2, features_2, target_2, cv=5)
final_score_2 = final_score = sum(scores_2) / len(scores_2)

df_valid_2 = pd.DataFrame(zip(target_valid_2))
df_valid_2 = df_valid_2.rename(columns={0: 'target_valid_2'})
df_valid_2["predictions_valid_2"] = predictions_valid_2

print("\nСредний запас предсказанного сырья:", float('{:.3f}'.format(predictions_valid_2.mean())))
print("RMSE модели model_2:", float('{:.3f}'.format(result_2)))
print("Средняя оценка качества модели:", float('{:.3f}'.format(final_score_2)))

print()
print(df_valid_0)
print(df_valid_1)
print(df_valid_2)


Средний запас предсказанного сырья: 92.549
RMSE модели model_0: 37.648
Средняя оценка качества модели: 0.275

Средний запас предсказанного сырья: 69.28
RMSE модели model_1: 0.895
Средняя оценка качества модели: 0.9996

Средний запас предсказанного сырья: 95.099
RMSE модели model_2: 40.128
Средняя оценка качества модели: 0.199

       target_valid_0  predictions_valid_0
0              145.83               123.28
1              134.02                75.84
2               88.91                55.53
3               19.51                86.67
4              108.71               109.91
...               ...                  ...
24995           80.10                89.71
24996           47.35                55.67
24997          176.38               139.04
24998          140.33                96.87
24999          117.32                54.71

[25000 rows x 2 columns]
       target_valid_1  predictions_valid_1
0               57.09                56.06
1               80.86                81.59

##### Вывод

- Получается, что чем больше запасы тем менее точное предсказание.
- До 180 RMSE считается нормальным и хорошим, так что все модели показали хороший результат, а model_1 вообще отличный.

## Подготовка к расчёту прибыли

- [x]  3.1. Все ключевые значения для расчётов сохраняем в отдельных переменных.
- [x]  3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.
- [x]  3.3. Напишем выводы по этапу подготовки расчёта прибыли.

In [ ]:
BUDGET = 10e+09              # бюджет на разработку скважин в регионе
PRODUCT_REVENUE = 450 * 1000 # доход с каждой единицы продукта
POINT = 200                  # точек для разработки в регионе

# Минимальный объём запасов в скважине (тыс. баррелей):
product_min = BUDGET / POINT / PRODUCT_REVENUE
print('\nМинимально необходимый объём запасов в скважине:', float('{:.3f}'.format(product_min)), 'тыс. баррелей')
print("Средний запас предсказанного сырья в регионе  0:", float('{:.3f}'.format(predictions_valid_0.mean())), 'тыс. баррелей')
print("Средний запас предсказанного сырья в регионе  1:", float('{:.3f}'.format(predictions_valid_1.mean())), 'тыс. баррелей')
print("Средний запас предсказанного сырья в регионе  2:", float('{:.3f}'.format(predictions_valid_2.mean())), 'тыс. баррелей')


Минимально необходимый объём запасов в скважине: 111.111 тыс. баррелей
Средний запас предсказанного сырья в регионе  0: 92.549 тыс. баррелей
Средний запас предсказанного сырья в регионе  1: 69.28 тыс. баррелей
Средний запас предсказанного сырья в регионе  2: 95.099 тыс. баррелей


##### Вывод

- По средним показателям выглядит не очень перспективно.

## Расчёт прибыли и рисков 

### Расчёт прибыли по выбранным скважинам и предсказаниям модели:

- [x]  4.1.1. Выбераем скважины с максимальными значениями предсказаний.
- [x]  4.1.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- [x]  4.1.3. Рассчитаем прибыль для полученного объёма сырья.

In [ ]:
'''Функция расчёта прибыли по выбранным скважинам и предсказаниям модели'''
def profit(target_x, predictions_x, count):
        predictions_sorted = predictions_x.sort_values(ascending=False) # сортируем предсказания
        selected = target_x[predictions_sorted.index][:count]           # выбираем ответы через их индексы
        return PRODUCT_REVENUE * selected.sum() - BUDGET                # прибыль для полученного объёма сырья
 
state = np.random.RandomState(12345)
count = 200

# df_0
target_valid_0 = df_valid_0['target_valid_0']           # в уникальную переменную, далее пригодится
predictions_valid_0 = df_valid_0['predictions_valid_0'] # в уникальную переменную, далее пригодится

target_0 = target_valid_0.sample(n=500, 
                                 replace=False, 
                                 random_state=state)
predictions_0 = predictions_valid_0[target_0.index] 
value_0 = profit(target_0, predictions_0, count)
print('\nПрибыль для полученного объёма сырья в регионе 0:', float('{:.2f}'.format(value_0)), 'рублей дохода')

# df_1
target_valid_1 = df_valid_1['target_valid_1']
predictions_valid_1 = df_valid_1['predictions_valid_1']

target_1 = target_valid_1.sample(n=500, 
                                 replace=False, 
                                 random_state=state)
predictions_1 = predictions_valid_1[target_1.index]
value_1 = profit(target_1, predictions_1, count)
print('Прибыль для полученного объёма сырья в регионе 1:', float('{:.2f}'.format(value_1)), 'рублей дохода')

# df_2
target_valid_2 = df_valid_2['target_valid_2']
predictions_valid_2 = df_valid_2['predictions_valid_2']
 
target_2 = target_valid_2.sample(n=500, 
                                 replace=False, 
                                 random_state=state)
predictions_2 = predictions_valid_2[target_2.index] 
value_2 = profit(target_2, predictions_2, count)
print('Прибыль для полученного объёма сырья в регионе 2:', float('{:.2f}'.format(value_2)), 'рублей дохода')
print()


Прибыль для полученного объёма сырья в регионе 0: 578185394.3 рублей дохода
Прибыль для полученного объёма сырья в регионе 1: 524087369.26 рублей дохода
Прибыль для полученного объёма сырья в регионе 2: 480270618.64 рублей дохода



### Риски и прибыль для каждого региона:

- [x]  5.1. Применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- [x]  5.2. Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- [x]  5.3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.

In [ ]:
# df_0
values_0 = []

for i in range(1000):                   # Bootstrap с 1000 выборок
    target_0 = target_valid_0.sample(n=500, replace=False, random_state=state)
    predictions_0 = predictions_valid_0[target_0.index]
    values_0.append(profit(target_0, predictions_0, count))

values_0 = pd.Series(values_0)
#print(values_0.sort_values())
lower_down_0 = values_0.quantile(.025)  # 95%-й доверительный интервал
lower_up_0 = values_0.quantile(.975)
mean_0 = values_0.mean()
print("\nСредняя выручка региона 0:", float('{:.2f}'.format(mean_0)), 'рублей дохода')
print("2,5% снизу:", float('{:.2f}'.format(lower_down_0)))
print("2,5% сверху:", float('{:.2f}'.format(lower_up_0)))
print('Вероятность убытков:', float('{:.2f}'.format(values_0[values_0 < 0].count() / 1000 * 100)),'%')


# df_1
values_1 = []

for i in range(1000):
    target_1 = target_valid_1.sample(n=500, replace=False, random_state=state)
    predictions_1 = predictions_valid_1[target_1.index]
    values_1.append(profit(target_1, predictions_1, count))

values_1 = pd.Series(values_1)
#print(values_1.sort_values())
lower_down_1 = values_1.quantile(.025)
lower_up_1 = values_1.quantile(.975)
mean_1 = values_1.mean()
print("\nСредняя выручка региона 1:", float('{:.2f}'.format(mean_1)), 'рублей дохода')
print("2,5% снизу:", float('{:.2f}'.format(lower_down_1)))
print("2,5% сверху:", float('{:.2f}'.format(lower_up_1)))
print('Вероятность убытков:', float('{:.2f}'.format(values_1[values_1 < 0].count() / 1000 * 100)),'%')


# df_2
values_2 = []

for i in range(1000):
    target_2 = target_valid_2.sample(n=500, replace=False, random_state=state)
    predictions_2 = predictions_valid_2[target_2.index]
    values_2.append(profit(target_2, predictions_2, count))

values_2 = pd.Series(values_2)
#print(values_2.sort_values())
lower_down_2 = values_2.quantile(.025)
lower_up_2 = values_2.quantile(.975)
mean_2 = values_2.mean()
print("\nСредняя выручка региона 2:", float('{:.2f}'.format(mean_2)), 'рублей дохода')
print("2,5% снизу:", float('{:.2f}'.format(lower_down_2)))
print("2,5% сверху:", float('{:.2f}'.format(lower_up_2)))
print('Вероятность убытков:', float('{:.2f}'.format(values_2[values_2 < 0].count() / 1000 * 100)),'%')


Средняя выручка региона 0: 489601919.05 рублей дохода
2,5% снизу: 5389429.38
2,5% сверху: 975341333.85
Вероятность убытков: 2.4 %

Средняя выручка региона 1: 460417069.86 рублей дохода
2,5% снизу: 78069470.7
2,5% сверху: 862284967.05
Вероятность убытков: 1.0 %

Средняя выручка региона 2: 337693169.45 рублей дохода
2,5% снизу: -198666391.27
2,5% сверху: 851403223.46
Вероятность убытков: 10.2 %


##### Вывод

- Средняя выручка региона 0: 487 млн. руб.
  С вероятностью 95% выручка может быть от 55 до 975 млн. руб.

- Средняя выручка региона 1: 460 млн. руб.
  С вероятностью 95% выручка может быть от 78 до 862 млн. руб.

- Средняя выручка региона 2: 340 млн. руб.
  С вероятностью 95% выручка может быть от -198 до 851 млн. руб.


    Для разработки стоило бы взять регион 1.
    Хотя средняя выручка там меньше на 5,5% чем в регионе 0. Но минимальная выручка на 41,8% больше, а вероятность убытков в 2,4 раза меньше.

    В регионе 2 с вероятностью 10,2% возможны убытки до 198 млн. руб.
    
    Но согласно условиям задачи "После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью." выбраным должен быть регион 0.